In [1]:
#conv Neural Network
#tensorboard --logdir=/home/ncc/notebook/learn/tensorboard/log
#save_and_restore =Not yet
#tensorboard = Not yet
"""
created by kim Seong jung , Medi-Whale.Inc 
"""

import numpy as np 
import tensorflow as tf
from PIL import Image
import math
import time
import matplotlib.pyplot as plt
%matplotlib inline
import os 
import math

file_locate='/home/ncc/Desktop/npy/'
sess = tf.InteractiveSession()
test_img=np.load(file_locate+'test_img.npy');

img_row =28
img_col =28
batch_size=30
print img_row ,img_col
n_classes =10
in_ch =1
out_ch1=20
out_ch2=20
out_ch3=20
out_ch4=20
out_ch5=20


fully_ch1=300
fully_ch2 =300
fully_ch3 =1024


############convolution layer setting#############
conv_weight_1 =[3,3,in_ch,out_ch1]
conv_weight_2 =[3,3,out_ch1,out_ch2]
conv_weight_3 =[3,3,out_ch2,out_ch3]
conv_weight_4 =[3,3,out_ch3,out_ch4]
conv_weight_5 =[3,3,out_ch4,out_ch5]

conv_strides_1=[1,1,1,1]
conv_strides_2=[1,1,1,1]
conv_strides_3=[1,1,1,1]
conv_strides_4=[1,1,1,1]
conv_strides_5=[1,1,1,1]

############pooling layer settting #############

pool_weight_1=[1,2,2,1]
pool_weight_2=[1,2,2,1]
pool_wieght_5=[1,2,2,1]

pool_strides_1=[1,2,2,1]
pool_strides_2=[1,2,2,1]
pool_strides_5=[1,2,2,1]


#######################################################################################
#                                   AlexNet                                           #
#                                                                                     #
# conv1 <K: 1,11,11,1> <S: 1,4,4,1>->lrn-->relu-> max pooling <K 1,3,3,1> ,<S 1,2,2,1>#
# conv2 <K: 1,5,5,1> <S: 1,1,1,1>->lrn-->relu-> max pooling <K 1,3,3,1> ,<S 1,2,2,1>  #
# conv3 <K: 1,3,3,1> <S: 1,1,1,1>                                                     #
# conv4 <K: 1,3,3,1> <S: 1,1,1,1>                                                     #
# conv5 <K: 1,3,3,1> <S: 1,1,1,1>->lrn->relu-> max pooling <K 1,3,3,1> ,<S 1,2,2,1>   #
# fc1-->matmul-->relu-->dropout                                                       #
# fc2-->matmul-->relu-->dropout                                                       #
# softmax-->y_conv                                                                    #
#######################################################################################




x= tf.placeholder("float",shape=[30,img_col*img_row *in_ch],  name = 'x-input')
y_=tf.placeholder("float",shape=[30 , n_classes] , name = 'y-input')



keep_prob = tf.placeholder("float")

x_image= tf.reshape(x,[30,img_row,img_col,in_ch])

iterate=30000




pooling_row_size1=int(img_row/2)
pooling_row_size2=int(pooling_row_size1/2)
pooling_row_size3=int(pooling_row_size2/2)
pooling_row_size4=int(pooling_row_size3/2)
pooling_row_size5=int(pooling_row_size4/2)
pooling_col_size1=int(img_col/2)
pooling_col_size2=int(pooling_col_size1/2)
pooling_col_size3=int(pooling_col_size2/2)
pooling_col_size4=int(pooling_col_size3/2)
pooling_col_size5=int(pooling_col_size4/2)

print img_col , img_row


28 28
28 28


In [2]:
with tf.device('/gpu:3'):
    #with tf.device('/gpu:1'):
    train_img=np.load(file_locate+'train_img.npy');
    train_lab=np.load(file_locate+'train_lab.npy');
    val_img= np.load(file_locate+'val_img.npy');
    val_lab = np.load(file_locate+'val_lab.npy');
    test_img=np.load(file_locate+'test_img.npy');
    test_lab=np.load(file_locate+'test_lab.npy');

    print "Training Data",np.shape(train_img)
    print "Training Data Label",np.shape(train_lab)
    print "Test Data Label",np.shape(test_lab)
    print "val Data Label" , np.shape(val_img)

    n_train= np.shape(train_img)[0]
    n_train_lab = np.shape(train_lab)[0]

Training Data (55000, 784)
Training Data Label (55000, 10)
Test Data Label (10000, 10)
val Data Label (5000, 784)


In [3]:
def aug_and_train(batch_x , batch_y , crop_row , crop_col ,keep_prob , sess , reflect = 'horizon' ):
    """
    
    image size is 256 ,256 
    
        for i in range(aug_num):
         for start_x in range(remainder_row):
        for start_y in range(remainder_col):
            print start_x , start_y
            cropped_img= img[start_x :start_x+width ,start_y:start_y+height,:]
            rev_cropped_img =np.fliplr(cropped_img)
            plt.imshow(new_img)
            img[start_x :start_x+width ,start_y:start_y+height,:]=rev_cropped_img
        sess.run(train_step ,feed_dict={x:batch_xs , y_:batch_ys , keep_prob : 0.7})
    np_pic shape have to [# , row , col , ch]
    """
    ori_img=np_pic
    size_batch,row ,col , ch = np.shape(ori_img)
    print size_batch, row ,col ,ch


    img=ori_img.copy()
    remainder_row=row-crop_row
    remainder_col=col-crop_col
    new_img = np.zeros([row-remainder_row, col-remainder_col, ch])
    print np.shape(new_img)
    #print img
    height = col -remainder_col
    width =  row - remainder_row
    i=0 ;j=0;
    start_time=time.time()
    for ind_batch in range(len(batch_x)):
        img_batch = batch_x[ind_batch:ind_batch+1 , : ,: ,: ]
        lab_batch = batch_y[ind_batch:ind_batch+1,  :, : ,: ]
        for start_x in range(remainder_row):
            for start_y in range(remainder_col):
                print start_x , start_y
                cropped_img= img[start_x :start_x+width ,start_y:start_y+height,:] #crop image that appointed  
                if reflect== 'horizon':
                    rev_cropped_img =np.fliplr(cropped_img) # horizontal reflection 
                img[start_x :start_x+width ,start_y:start_y+height,:]=rev_cropped_img # attach image                      
                sess.run(train_step ,feed_dict={x:img, y_:lab , keep_prob : 0.5}) #training imgae 
                end_time = time.time()
        total_time=end_time-start_time
        print total_time     



In [4]:
"""def weight_variable(name,shape):
    #initial = tf.truncated_normal(shape , stddev=0.1)
    initial = tf.get_variable(name,shape=shape , initializer = tf.contrib.layers.xavier_initializer())
    return tf.Variable(initial)"""
with tf.device('/gpu:0'):
    def bias_variable(shape):
        initial = tf.constant(0.1 , shape=shape)
        return tf.Variable(initial)



In [5]:
with tf.device('/gpu:0'):
    def next_batch(batch_size , image , label):

        a=np.random.randint(np.shape(image)[0] -batch_size)
        batch_x = image[a:a+batch_size,:]
        batch_y= label[a:a+batch_size,:]
        return batch_x, batch_y


In [6]:
with tf.device('/gpu:0'):

    def conv2d(x,w,conv_strides_):
        return tf.nn.conv2d(x,w, strides = conv_strides_, padding='SAME')
    def max_pool(x , ksize=[1,2,2,1] ,  strides = [1,2,2,1], padding = 'SAME'):
        return tf.nn.max_pool(x , ksize ,strides, padding)

In [7]:
def local_response_nomalization(conv , radius , alpha , beta , bias , batch_size =30):
    
    """
    in paper , alexent 
    hyperparameter  n=5, alpha= 10^-4 , beta=0.75 , bias = 2 
    
    sqr_sum[a, b, c, d] =
    sum(input[a, b, c, d - depth_radius : d + depth_radius + 1] ** 2)
    output = input / (bias + alpha * sqr_sum) ** beta
    
    if you don't understand above code look at the code at below
    #############################################################
    a=[1,2,3,4,]
    a=np.ndarray(shape=(2,3))
    a[0,0]=1;a[0,1]=2;a[0,2]=3 ;a[1,0]=4;a[1,1]=5;a[1,2]=6
    
    
    c=(a[1,:3] **2 )
    print a    
    print c
    [[ 1.  2.  3.]
     [ 4.  5.  6.]]
    [ 16.  25.  36.]
    #############################################################
    """

    
    
    if type(conv).__module__ == np.__name__:
        N , row , col , n_ch =np.shape(conv)
        lrn_conv = np.zeros(shape=[N,row ,col , n_ch])
        for i in range(N):
            for r in range(row):
                for c in range(col):
                    #conv[row][col] in paper , Alex == a(x,y) (at apointed kernel i )
                    max_range =max(0,c - radius/2)
                    min_range =min(n_ch-1,c+radius/2)       
                    #if c==0:   
                    #print max_range , min_range
                    
                    for ch in range(max_range,min_range):
                        
                        normalized_constant=(bias+alpha*(sum(conv[i,r,c,min_range : max_range])**2))**beta
                        lrn_img=conv[i,:,:,:]/normalized_constant
                        lrn_conv[i,:,:,:]=lrn_img 

    return lrn_conv



In [8]:
a=np.ndarray(shape=[2,2])
print type(a).__module__ =='numpy'

True


In [9]:

import matplotlib.pyplot as plt
from PIL import Image 
path ='/home/ncc/Desktop/image/'

def make_dummy_img(path):
    paths=os.listdir(path)
    ret_img = np.zeros([len(paths) , 256,256,3])
    
    for i in range(len(paths)):
        img=Image.open(path+paths[i])
        img=np.array(img)
        ret_img[i , :,:,:] = img



    return ret_img
conv= make_dummy_img(path)
print np.shape(conv)
print type(conv)
radius=2 ;  alpha= 0.0004  ; beta=0.75 ;  bias = 2 
lrn_conv=local_response_nomalization(conv , radius , alpha , beta , bias)
#print result 
print conv
print lrn_conv

(10, 256, 256, 3)
<type 'numpy.ndarray'>
[[[[  23.   23.   23.]
   [  24.   24.   24.]
   [  24.   24.   24.]
   ..., 
   [  26.   26.   26.]
   [  22.   22.   22.]
   [  29.   29.   29.]]

  [[  17.   17.   17.]
   [  17.   17.   17.]
   [  18.   18.   18.]
   ..., 
   [  17.   17.   15.]
   [  20.   20.   20.]
   [  14.   14.   12.]]

  [[ 197.  197.  197.]
   [ 197.  197.  197.]
   [ 197.  197.  197.]
   ..., 
   [ 204.  203.  199.]
   [ 202.  201.  199.]
   [ 165.  164.  160.]]

  ..., 
  [[  27.   24.   19.]
   [  21.   20.   15.]
   [  31.   28.   23.]
   ..., 
   [  25.   26.   28.]
   [  24.   26.   25.]
   [  23.   25.   24.]]

  [[  23.   22.   20.]
   [  28.   28.   26.]
   [  24.   23.   21.]
   ..., 
   [  27.   27.   29.]
   [  25.   25.   25.]
   [  22.   22.   22.]]

  [[  25.   25.   25.]
   [  23.   23.   23.]
   [  20.   20.   20.]
   ..., 
   [  21.   21.   23.]
   [  24.   22.   23.]
   [  27.   25.   26.]]]


 [[[ 100.   80.   53.]
   [ 101.   81.   54.]
   [ 101.

In [10]:
a=[1,2,3,4,]
a=np.ndarray(shape=(2,3))
a[0,0]=1;a[0,1]=2;a[0,2]=3 ;a[1,0]=4;a[1,1]=5;a[1,2]=6

print a
c=(a[1,:3] **2 )
print c
d=sum(c)
print d
print a/d

[[ 1.  2.  3.]
 [ 4.  5.  6.]]
[ 16.  25.  36.]
77.0
[[ 0.01298701  0.02597403  0.03896104]
 [ 0.05194805  0.06493506  0.07792208]]


In [11]:
with tf.variable_scope("layer1") as scope:
    try:
        w_conv1 = tf.get_variable("W1",conv_weight_1 , initializer = tf.contrib.layers.xavier_initializer())
    except:
        scope.reuse_variables()
        w_conv1 = tf.get_variable("W1",conv_weight_1 , initializer = tf.contrib.layers.xavier_initializer())
with tf.variable_scope("layer1") as scope:
    try:
        b_conv1 = bias_variable([out_ch1])
    except:
        scope.reuse_variables()
        b_conv1 = bias_variable([out_ch1])
                
            
            
with tf.variable_scope('layer2') as scope:
    try:
        w_conv2 = tf.get_variable("W2",conv_weight_2 , initializer = tf.contrib.layers.xavier_initializer())
    except:
        scope.reuse_variables()
        w_conv2 = tf.get_variable("W2",conv_weight_2 , initializer = tf.contrib.layers.xavier_initializer())
        
with tf.variable_scope('layer2') as scope:
    try:
        b_conv2= bias_variable([out_ch2])
    except:
        scope.reuse_variables()
        b_conv2= bias_variable([out_ch2])
                
with tf.variable_scope('layer3') as scope:
    try:
        w_conv3 = tf.get_variable("W3" ,conv_weight_3 , initializer = tf.contrib.layers.xavier_initializer())
    except:
        scope.reuse_variables()
        w_conv3 = tf.get_variable("W3" ,conv_weight_3 , initializer = tf.contrib.layers.xavier_initializer())
with tf.variable_scope('layer3') as scope:
    try:
        b_conv3 = bias_variable([out_ch3])
    except:
        scope.reuse_variables()
        b_conv3 = bias_variable([out_ch3])
        
with tf.variable_scope('layer4') as scope:
    try:
        w_conv4 =tf.get_variable("W4" ,conv_weight_4 , initializer = tf.contrib.layers.xavier_initializer())
    except:
        scope.reuse_variables()
        w_conv3 = tf.get_variable("W4" ,conv_weight_4 , initializer = tf.contrib.layers.xavier_initializer())
with tf.variable_scope('layer4') as scope:
    try:
        b_conv4 = bias_variable([out_ch4])
    except:
        scope.reuse_variables()
        b_conv3 = bias_variable([out_ch3])
        
with tf.variable_scope('layer5') as scope:
    try:
        w_conv5 = tf.get_variable("W5",conv_weight_5 , initializer = tf.contrib.layers.xavier_initializer())
    except:
        scope.reuse_variables()
        w_conv3 = tf.get_variable("W5" ,conv_weight_5 , initializer = tf.contrib.layers.xavier_initializer())
with tf.variable_scope('layer5') as scope:
    try:
        b_conv5 = bias_variable([out_ch5])
    except:
        scope.reuse_variables()
        b_conv3 = bias_variable([out_ch3])
                

In [12]:
#conncect hidden layer 
with tf.device('/gpu:0'):
     
    h_conv1 = tf.nn.relu(conv2d(x_image , w_conv1 ,conv_strides_1)+b_conv1)
    assert len(h_conv1.get_shape()[1:]) ==3 , h_conv1.get_shape()[1:]   
    assert out_ch1==h_conv1.get_shape()[-1] , out_ch1   
    row_h1 , col_h1 , ch_h1 =h_conv1.get_shape()[1:]
    x_h1 = tf.placeholder(tf.float32,shape=[30,row_h1 , col_h1 , ch_h1],  name = 'layer1_lrn')
    
    #h_conv1_lrn = local_response_nomalization(h_conv1 , radius=2 , alpha=2e-05 , beta = 0.75 , bias = 1.0)
    #논리적으로는 h_conv1 뒤에 lrn 이 나오지만 tensorflow 구조상 
    #여기에는 보이지 않는다 .  아래 training 구문에서  에서는 실재로 lrn 코드가 여기 들어간다 
    h_conv1_max = max_pool(x_h1)#pooling
 
    h_conv2 = tf.nn.relu(conv2d(h_conv1_max , w_conv2 ,conv_strides_2)+b_conv2)
    assert len(h_conv2.get_shape()[1:])==3  ,h_conv2.get_shape()[1:] 
    assert h_conv2.get_shape()[-1] == out_ch2 ,out_ch2
    row_h2 , col_h2 , ch_h2 =h_conv2.get_shape()[1:]
    
    #h_conv2_lrn = local_response_nomalization(h_conv2 , radius=2 , alpha=2e-05 , beta = 0.75 , bias = 1.0)    
    x_h2 = tf.placeholder( tf.float32 ,shape=[30,row_h2 , col_h2 , ch_h2],  name = 'layer2_lrn')
    h_conv2_max = max_pool(x_h2 , pool_weight_2 , pool_strides_2)#pooling

    h_conv3 = tf.nn.relu(conv2d(h_conv2_max , w_conv3,conv_strides_3)+b_conv3)
    
    h_conv4 = tf.nn.relu(conv2d(h_conv3 , w_conv4,conv_strides_4)+b_conv4)
#x_h5 = tf.placeholder("float",shape=[None,img_col , img_row , 3],  name = 'layer3_lrn')
    
    h_conv5 = tf.nn.relu(conv2d(h_conv4, w_conv5,conv_strides_5)+b_conv5)
    #h_conv5_lrn = local_response_nomalization(h_conv5 , radius=2 , alpha=2e-05 , beta = 0.75 , bias = 1.0)
    row_h5 , col_h5 , ch_h5 =h_conv5.get_shape()[1:] 
    
    x_h5 = tf.placeholder(tf.float32 ,shape=[30,row_h5 , col_h5 , ch_h5],  name = 'layer5_lrn')
    assert len(h_conv5.get_shape()[1:])==3  ,h_conv5.get_shape()[1:] 
    assert h_conv5.get_shape()[-1] == out_ch5 , h_conv5.get_shape[-1]  
    h_conv5_max = max_pool(x_h5)
#
print h_conv1
print h_conv2
print h_conv3
print h_conv4
print h_conv5

    
end_conv = h_conv5_max
    #print conv2d(h_pool1 , w_conv2).get_shape()


Tensor("Relu:0", shape=(30, 28, 28, 20), dtype=float32, device=/device:GPU:0)
Tensor("Relu_1:0", shape=(30, 14, 14, 20), dtype=float32, device=/device:GPU:0)
Tensor("Relu_2:0", shape=(30, 7, 7, 20), dtype=float32, device=/device:GPU:0)
Tensor("Relu_3:0", shape=(30, 7, 7, 20), dtype=float32, device=/device:GPU:0)
Tensor("Relu_4:0", shape=(30, 7, 7, 20), dtype=float32, device=/device:GPU:0)


In [13]:
print w_conv1.get_shape()


(3, 3, 1, 20)


In [14]:
end_conv_row=int(h_conv5_max.get_shape()[1])
end_conv_col=int(h_conv5_max.get_shape()[2])
end_conv_ch=int(h_conv5_max.get_shape()[3])

print end_conv_row , end_conv_col , end_conv_ch
#connect fully connected layer 
with tf.device('/gpu:0'):
    with tf.variable_scope("fc1") as scope:
        try:
            w_fc1=tf.get_variable("fc1_W",[end_conv_col*end_conv_row*end_conv_ch,fully_ch1] , initializer = tf.contrib.layers.xavier_initializer())
        except:
            scope.reuse_variables()
            w_fc1=tf.get_variable("fc1_W",[end_conv_col*end_conv_row*end_conv_ch,fully_ch1] , initializer = tf.contrib.layers.xavier_initializer())
        try:
            b_fc1 = bias_variable([fully_ch1])
        except:
            scope.reuse_variables()
            b_fc1 = bias_variable([fully_ch1])

        
with tf.device('/gpu:0'): # flat conv layer 
    end_flat_conv =tf.reshape(end_conv, [-1,end_conv_col*end_conv_row*end_conv_ch])
   
with tf.device('/gpu:0'): # connect flat layer with fully  connnected layer 
    h_fc1 = tf.nn.relu(tf.matmul(end_flat_conv , w_fc1)+ b_fc1)
    h_fc1 = tf.nn.dropout(h_fc1, keep_prob)

4 4 20


In [15]:
with tf.device('/gpu:0'):
    with tf.variable_scope('fc2') as scope:
        try:
            w_fc2 =tf.get_variable("fc2_W",[fully_ch1 , fully_ch2],initializer = tf.contrib.layers.xavier_initializer())
        except:
            scope.reuse_variables()
            w_fc2 =tf.get_variable("fc2_W",[fully_ch1 , fully_ch2],initializer = tf.contrib.layers.xavier_initializer())
        try:
            b_fc2 = bias_variable([fully_ch2])
        except:
            scope.reuse_variables()
            b_fc2 = bias_variable([fully_ch2])

with tf.device('/gpu:0'):  # join flat layer with fully  connnected layer 
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1 , w_fc2)+b_fc2)
    h_fc2= tf.nn.dropout(h_fc2 , keep_prob)
    

In [16]:
end_fc=h_fc2
end_ch=fully_ch2

In [17]:
with tf.device('/gpu:0'):
    with tf.variable_scope('end') as scope:
        try:
            w_end =tf.get_variable("end_W",[end_ch, n_classes ],initializer = tf.contrib.layers.xavier_initializer())
        except:
            scope.reuse_variables()
            w_end =tf.get_variable("end_W",[end_ch , n_classes],initializer = tf.contrib.layers.xavier_initializer())
        try:
            b_end = bias_variable([n_classes])
        except:
            scope.reuse_variables()
            b_end = bias_variable([n_classes])

with tf.device('/gpu:0'):  # join flat layer with fully  connnected layer 
    y_conv = tf.nn.relu(tf.matmul(end_fc , w_end)+b_end)
    

In [18]:
#dirname = '/home/ncc/notebook/mammo/result/'

dirname='/home/ncc/Desktop/result/'

count=0
while(True):
    if not os.path.isdir(dirname):
        os.mkdir(dirname)
        break
    elif not os.path.isdir(dirname + str(count)):
        dirname=dirname+str(count)
        os.mkdir(dirname)
        break
    else:
        count+=1
print 'it is recorded at :'+str(count)

it is recorded at :2


In [19]:
f=open(dirname+"/log.txt",'w')

In [20]:
with tf.device('/gpu:0'):
#sm_conv= tf.nn.softmax(y_conv)
    #cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
    start_time = time.time()

    regular=0.01*(tf.reduce_sum(tf.square(y_conv)))
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits( y_conv, y_))
with tf.device('/gpu:0'):
    cost = cost+regular
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cost) #1e-4
    with tf.name_scope("accuracy"):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.equal(tf.argmax(y_conv,1) ,tf.argmax(y_,1))
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction , "float")) 

    sess = tf.Session()
    sess.run(tf.initialize_all_variables())

for i in range(iterate):
    
    batch_xs , batch_ys = next_batch(batch_size, train_img , train_lab)
   # batch_val_xs  , batch_val_ys = next_batch(20 , val_img , val_lab)
    if i%100 ==0: # in here add to validation 
        try:
            np_h_conv1=sess.run(h_conv1 , feed_dict={x: val_img[:10], y_ : val_lab[:10] , keep_prob : 0.5})
            np_h_conv1_lrn = local_response_nomalization(np_h_conv1 , radius , alpha , beta , bias , 'layer1' , batch_size)
            np_h_conv1_lrn=np_h_conv1_lrn.astype(np.float32)

            np_h_conv2=sess.run(h_conv2, feed_dict = {x_h1:np_h_conv1_lrn, y_:batch_ys , keep_prob : 0.5}  )
            np_h_conv2_lrn = local_response_nomalization(np_h_conv2 , radius , alpha , beta , bias , 'layer2' , batch_size)
            np_h_conv2_lrn=np_h_conv2_lrn.astype(np.float32)

            np_h_conv5=sess.run(h_conv5, feed_dict = {x_h2:np_h_conv1_lrn, y_:batch_ys , keep_prob : 0.5}  )
            np_h_conv5_lrn = local_response_nomalization(np_h_conv5 , radius , alpha , beta , bias , 'layer5' , batch_size)
            np_h_conv5_lrn=np_h_conv5_lrn.astype(np.float32) 

            val_accuracy = sess.run( accuracy , feed_dict={x_h5:np_h_conv5_lrn , y_:val_lab , keep_prob: 1.0})        
            val_loss = sess.run(cost , feed_dict = {x_h5:np_h_conv5_lrn , y_: val_lab , keep_prob: 1.0})
            """
            np_h_conv1=sess.run(h_conv1 , feed_dict={x: batch_xs, y_ : keep_prob , keep_prob : 0.5})
            np_h_conv1_lrn = local_response_nomalization(np_h_conv1 , radius , alpha , beta , bias , 'layer1' , batch_size)
            np_h_conv1_lrn=np_h_conv1_lrn.astype(float32)

            np_h_conv2=sess.run(h_conv2, feed_dict = {x2:np_h_conv1_lrn, y_:batch_ys , keep_prob : 0.5}  )
            np_h_conv2_lrn = local_response_nomalization(np_h_conv2 , radius , alpha , beta , bias , 'layer2' , batch_size)
            np_h_conv2_lrn=np_h_conv2_lrn.astype(np.float32)

            np_h_conv5=sess.run(h_conv5, feed_dict = {x5:np_h_conv1_lrn, y_:batch_ys , keep_prob : 0.5}  )
            np_h_conv5_lrn = local_response_nomalization(np_h_conv5 , radius , alpha , beta , bias , 'layer5' , batch_size)
            np_h_conv5_lrn=np_h_conv5_lrn.astype(np.float32) 

            
            train_accuracy = sess.run( accuracy , feed_dict={x:np_h_conv5_lrn , y_:batch_ys , keep_prob: 1.0})        
            train_loss = sess.run(cost , feed_dict = {x:np_h_conv5_lrn, y_: batch_ys, keep_prob: 1.0})
            
            #result = sess.run(sm_conv , feed_dict = {x:val_img , y_:batch_ys , keep_prob :1.0})
            print("step %d , training  accuracy %g" %(i,train_accuracy))
            print("step %d , loss : %g" %(i,train_loss))
            train_str = 'step:\t'+str(i)+'\tval_loss:\t'+str(train_loss) +'\tval accuracy:\t'+str(train_accuracy)+'\n'
            """
            print("step %d , validation  accuracy %g" %(i,val_accuracy))
            print("step %d , validation loss : %g" %(i,val_loss))
            val_str = 'step:\t'+str(i)+'\tval_loss:\t'+str(val_loss) +'\tval accuracy:\t'+str(val_accuracy)+'\n'
            
            
            f.write(val_str)
            #f.write(train_str)
        except :
            list_acc=[]
            list_loss=[]
            n_divide=len(val_img)/batch_size
            for j in range(n_divide):
            
            
            #x:val_img[ j*batch_size :(j+1)*batch_size] , y_:val_lab[ j*batch_size :(j+1)*batch_size ] , keep_prob: 1.0
                # j*batch_size :(j+1)*batch_size
             
                np_h_conv1=sess.run(h_conv1 , feed_dict={x: val_img[j*batch_size :(j+1)*batch_size], y_ : val_lab[j*batch_size :(j+1)*batch_size] , keep_prob : 0.5})
                #print h_conv1.get_shape()
                #print np.shape(np_h_conv1)
                np_h_conv1_lrn = local_response_nomalization(np_h_conv1 , radius , alpha , beta , bias  , batch_size)
                np_h_conv1_lrn=np_h_conv1_lrn.astype(np.float32)
                
                #print np.shape(np_h_conv1_lrn)
                
                np_h_conv2=sess.run(h_conv2, feed_dict = {x_h1:np_h_conv1_lrn, y_:batch_ys , keep_prob : 0.5}  )
                np_h_conv2_lrn = local_response_nomalization(np_h_conv2 , radius , alpha , beta , bias  , batch_size)
                np_h_conv2_lrn=np_h_conv2_lrn.astype(np.float32)

                np_h_conv5=sess.run(h_conv5, feed_dict = {x_h2:np_h_conv2_lrn, y_:batch_ys , keep_prob : 0.5}  )
                np_h_conv5_lrn = local_response_nomalization(np_h_conv5 , radius , alpha , beta , bias  , batch_size)
                np_h_conv5_lrn=np_h_conv5_lrn.astype(float) 
                #print np.shape(np_h_conv5_lrn)

                val_accuracy = sess.run( accuracy , feed_dict={x_h5:np_h_conv5_lrn ,y_ : val_lab[j*batch_size :(j+1)*batch_size] , keep_prob: 1.0})        
                val_loss = sess.run(cost , feed_dict = {x_h5:np_h_conv5_lrn , y_: val_lab[j*batch_size :(j+1)*batch_size], keep_prob: 1.0 })
                list_acc.append(float(val_accuracy))
                list_loss.append(float(val_loss))

            list_acc=np.asarray(list_acc)
            list_loss= np.asarray(list_loss)
            
            val_accuracy=np.mean(list_acc)
            val_loss = np.mean(list_loss)
            
            #result = sess.run(sm_conv , feed_dict = {x:val_img , y_:batch_ys , keep_prob :1.0})
            """
            train_accuracy = sess.run( accuracy , feed_dict={x:batch_xs , y_:batch_ys , keep_prob: 1.0})        
            train_loss = sess.run(cost , feed_dict = {x:batch_xs, y_: batch_ys, keep_prob: 1.0})

            print("step %d , training  accuracy %g" %(i,train_accuracy))
            print("step %d , loss : %g" %(i,train_loss))
            train_str = 'step:\t'+str(i)+'\tval_loss:\t'+str(train_loss) +'\tval accuracy:\t'+str(train_accuracy)+'\n'
            """
            print("step %d , validation  accuracy %g" %(i,val_accuracy))
            print("step %d , validation loss : %g" %(i,val_loss))
            val_str = 'step:\t'+str(i)+'\tval_loss:\t'+str(val_loss) +'\tval accuracy:\t'+str(val_accuracy)+'\n'
           
            
            f.write(val_str)
            #f.write(train_str)
    
    #training
    np_h_conv1=sess.run(h_conv1 , feed_dict={x: batch_xs, y_ : batch_ys , keep_prob : 0.5})
    np_h_conv1_lrn = local_response_nomalization(np_h_conv1 , radius , alpha , beta , bias  , batch_size)
    np_h_conv1_lrn=np_h_conv1_lrn.astype(np.float32)
    
    np_h_conv2=sess.run(h_conv2, feed_dict = {x_h1:np_h_conv1_lrn, y_:batch_ys , keep_prob : 0.5}  )
    np_h_conv2_lrn = local_response_nomalization(np_h_conv2 , radius , alpha , beta , bias  , batch_size)
    np_h_conv2_lrn=np_h_conv2_lrn.astype(np.float32)
    
    np_h_conv5=sess.run(h_conv5, feed_dict = {x_h2:np_h_conv2_lrn, y_:batch_ys , keep_prob : 0.5}  )
    np_h_conv5_lrn = local_response_nomalization(np_h_conv5 , radius , alpha , beta , bias  , batch_size)
    np_h_conv5_lrn=np_h_conv5_lrn.astype(np.float32) 
    
    sess.run(train_step, feed_dict = {x_h5:np_h_conv5_lrn, y_:batch_ys , keep_prob : 0.5}  )
        
    
    #aug_and_train(batch_x , batch_y , crop_row , crop_col ,keep_prob =0.5 , sess , reflect = 'horizon' ):
print("--- Training Time : %s ---" % (time.time() - start_time))
train_time="--- Training Time : ---:\t" +str(time.time() - start_time)
f.write(train_time)

step 0 , validation  accuracy 0.10743
step 0 , validation loss : 2.42361
step 100 , validation  accuracy 0.0955823
step 100 , validation loss : 2.30259
step 200 , validation  accuracy 0.0955823
step 200 , validation loss : 2.30259
step 300 , validation  accuracy 0.0955823
step 300 , validation loss : 2.30259
step 400 , validation  accuracy 0.0955823
step 400 , validation loss : 2.30259
step 500 , validation  accuracy 0.0955823
step 500 , validation loss : 2.30259


KeyboardInterrupt: 

In [ ]:
def local_response_nomalization(conv , radius , alpha , beta , bias ,layer_name , batch_size =30 , ):
    

In [ ]:
try:
    test_accuracy = sess.run( accuracy , feed_dict={x:test_img , y_:test_lab , keep_prob: 1.0})        
    test_loss = sess.run(cost , feed_dict = {x:test_img , y_: test_lab , keep_prob: 1.0})

    #result = sess.run(sm_conv , feed_dict = {x:test_img , y_:batch_ys , keep_prob :1.0})
    print("step %d , testidation  accuracy %g" %(i,test_accuracy))
    print("step %d , testidation loss : %g" %(i,test_loss))
    test_str = 'step:\t'+str(i)+'\ttest_loss:\t'+str(test_loss) +'\ttest accuracy:\t'+str(test_accuracy)+'\n'

    f.write(test_str)
except :
    list_acc=[]
    list_loss=[]
    n_divide=len(test_img)/batch_size
    for j in range(n_divide):

        # j*batch_size :(j+1)*batch_size
        test_accuracy,test_loss = sess.run([accuracy ,cost], feed_dict={x:test_img[ j*batch_size :(j+1)*batch_size] , y_:test_lab[ j*batch_size :(j+1)*batch_size ] , keep_prob: 1.0})        
        list_acc.append(float(test_accuracy))
        list_loss.append(float(test_loss))
    test_accuracy , test_loss=sess.run([accuracy,cost] , feed_dict={x:test_img[(j+1)*batch_size : ] , y_:test_lab[(j+1)*(batch_size) : ] , keep_prob : 1.0})
    #right above code have to modify

    list_acc.append(test_accuracy)
    list_loss.append(test_loss)
    list_acc=np.asarray(list_acc)
    list_loss= np.asarray(list_loss)

    test_accuracy=np.mean(list_acc)
    test_loss = np.mean(list_loss)

    #result = sess.run(sm_conv , feed_dict = {x:test_img , y_:batch_ys , keep_prob :1.0})
    print("step %d , testidation  accuracy %g" %(i,test_accuracy))
    print("step %d , testidation loss : %g" %(i,test_loss))
    test_str = 'step:\t'+str(i)+'\ttest_loss:\t'+str(test_loss) +'\ttest accuracy:\t'+str(test_accuracy)+'\n'

    f.write(test_str)


In [ ]:
sess.close()
